In [1]:
import os
from sage.manifolds.utilities import ExpressionNice as EN

In [2]:
def formal_diff(f, x):
    if f!=0:
        tempX = SR.temp_var()
        return f.subs({x: tempX}).diff(tempX).subs({tempX: x})
    if f==0:
        return 0

In [3]:
#Definimos variables y funciones
var('eps', latex_name = '\\varepsilon')
var('t')
var('r')
var('th', latex_name = '\\theta')
var('ph', latex_name = '\\varphi')
coords = (t,r,th,ph)

h = function('h')(r,th)
k = function('k')(r,th)
v = function('v')(r,th)
omega = function('omega', latex_name='\\omega')(r)

nu = function('nu', latex_name='\\nu')(r)
lamb = function('lamb', latex_name='\\lambda')(r)

Ne = e^(nu/2)*(1+1/2*eps^2*h)
Ve = e^(lamb)*(1+eps^2*v)
Ke = r^2*(1+eps^2*k)
Nphie = eps*omega

# <center> Tensor de Einstein </center>

In [4]:
gdd = matrix(SR, 4, 4)

gdd[0,0] = -exp(nu)
gdd[1,1] = exp(lamb)
gdd[2,2] = r^2
gdd[3,3] = r^2*sin(th)^2

guu = 1/gdd

K1 = matrix(SR, 4, 4)
K1[0,3] = -omega*r^2*sin(th)^2
K1[3,0] = -omega*r^2*sin(th)^2

K2 = matrix(SR, 4, 4)
K2[0,0] = -4*exp(nu)*h + 2*r^2*sin(th)^2*omega^2
K2[1,1] = 4*exp(lamb)*v
K2[2,2] = 4*k*r^2
K2[3,3] = 4*k*r^2*sin(th)^2

gedd = matrix(SR, 4, 4)
for ii in range(4):
    for jj in range(4):
        gedd[ii,jj] = gdd[ii,jj] + eps*K1[ii,jj] + 1/2*eps^2*K2[ii,jj]
        
geuu = 1/gedd
#geuu = geuu0.subs(eps==0) + eps*geuu0

# <center> Tensor de energía-momento </center>

## <center> Campo $u$ de los neutrones </center>

In [5]:
Omegane = function('Omegane', latex_name='\\Omega_{n\\varepsilon}')(eps)

Nne_paper = 1/sqrt(Ne^2-sin(th)^2*Ke*(Nphie-eps*Omegane)^2)
ueu = [Nne_paper,0,0,eps*Nne_paper*Omegane]

ued = [0,0,0,0]
for ii in range(4):
    for jj in range(4):
        ued[ii] += ueu[jj]*gedd[ii,jj]

## <center> Campo $v$ de los protones </center>

In [6]:
Omegape = function('Omegape', latex_name='\\Omega_{p\\varepsilon}')(eps)

Npe_paper = 1/sqrt(Ne^2-sin(th)^2*Ke*(Nphie-eps*Omegape)^2)
veu = [Npe_paper,0,0,eps*Npe_paper*Omegape]

ved = [0,0,0,0]
for ii in range(4):
    for jj in range(4):
        ved[ii] += veu[jj]*gedd[ii,jj]

## <center> Ecuación de Einstein </center>

In [7]:
n = function('n')(r,th,eps)
p = function('p')(r,th,eps)
x = function('x')(r,th,eps)

Lambdae = function('Lambdae', latex_name='\\Lambda_{\\varepsilon}')(n, p, x)

neu = [n*ii for ii in ueu]
peu = [p*ii for ii in veu]

ned = [n*ii for ii in ued]
ped = [p*ii for ii in ved]

A = -formal_diff(Lambdae, x)
B = -2*formal_diff(Lambdae, n)
C = -2*formal_diff(Lambdae, p)

mued1 = [B*ii for ii in ned]
mued2 = [A*jj for jj in ped]
chied1 = [C*ii for ii in ped]
chied2 = [A*jj for jj in ned]

mued = [0,0,0,0]
chied = [0,0,0,0]
for ii in range(4):
    mued[ii] = mued1[ii] + mued2[ii]
    chied[ii] = chied1[ii] + chied2[ii]

Psie = Lambdae - sum([ii*jj for (ii,jj) in zip(neu, mued)]) - sum([kk*ll for (kk,ll) in zip(peu, chied)])

In [8]:
dirac = diagonal_matrix([1,1,1,1])
Teud = matrix(SR, 4, 4)
for ii in range(4):
    for jj in range(4):
        Teud[ii,jj] = Psie*dirac[ii,jj] + peu[ii]*chied[jj] + neu[ii]*mued[jj]

Te, dTe = matrix(SR, 4, 4), matrix(SR, 4, 4)
for ii in range(4):
    for jj in range(4):
        for kk in range(4):
            Te[ii,jj] += Teud[kk,jj]*gedd[kk,ii]

        dTe[ii,jj] = formal_diff(Te[ii,jj],eps)

In [13]:
show(EN(dTe[0,3].subs(eps==0).simplify_full()))

(2*r^2*Omegane(0)*n(r, th, 0)^2*d(Lambdae)/d(n(r, th, 0)) + 2*r^2*Omegape(0)*p(r, th, 0)^2*d(Lambdae)/d(p(r, th, 0)) + r^2*(Omegane(0) + Omegape(0))*n(r, th, 0)*p(r, th, 0)*d(Lambdae)/d(x(r, th, 0)) - r^2*Lambdae(n(r, th, 0), p(r, th, 0), x(r, th, 0))*omega(r))*sin(th)^2